### This notebook successfully imports the xgboost model from github

In [ ]:
import pandas as pd
import xgboost
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import roc_auc_score
import re

# These imports are used to load the json file into the XGBoost. They not used to for any predictions.
import requests
import json
import tempfile

In [3]:
# defining stop words. these stop words were taken from the spacy and nltk library
spacy = ['call', 'upon', 'still', 'nevertheless', 'down', 'every', 'forty', '‘re', 'always', 'whole', 'side', "n't", 'now', 'however', 'an', 'show', 'least', 'give', 'below', 'did', 'sometimes', 'which', "'s", 'nowhere', 'per', 'hereupon', 'yours', 'she', 'moreover', 'eight', 'somewhere', 'within', 'whereby', 'few', 'has', 'so', 'have', 'for', 'noone', 'top', 'were', 'those', 'thence', 'eleven', 'after', 'no', '’ll', 'others', 'ourselves', 'themselves', 'though', 'that', 'nor', 'just', '’s', 'before', 'had', 'toward', 'another', 'should', 'herself', 'and', 'these', 'such', 'elsewhere', 'further', 'next', 'indeed', 'bottom', 'anyone', 'his', 'each', 'then', 'both', 'became', 'third', 'whom', '‘ve', 'mine', 'take', 'many', 'anywhere', 'to', 'well', 'thereafter', 'besides', 'almost', 'front', 'fifteen', 'towards', 'none', 'be', 'herein', 'two', 'using', 'whatever', 'please', 'perhaps', 'full', 'ca', 'we', 'latterly', 'here', 'therefore', 'us', 'how', 'was', 'made', 'the', 'or', 'may', '’re', 'namely', "'ve", 'anyway', 'amongst', 'used', 'ever', 'of', 'there', 'than', 'why', 'really', 'whither', 'in', 'only', 'wherein', 'last', 'under', 'own', 'therein', 'go', 'seems', '‘m', 'wherever', 'either', 'someone', 'up', 'doing', 'on', 'rather', 'ours', 'again', 'same', 'over', '‘s', 'latter', 'during', 'done', "'re", 'put', "'m", 'much', 'neither', 'among', 'seemed', 'into', 'once', 'my', 'otherwise', 'part', 'everywhere', 'never', 'myself', 'must', 'will', 'am', 'can', 'else', 'although', 'as', 'beyond', 'are', 'too', 'becomes', 'does', 'a', 'everyone', 'but', 'some', 'regarding', '‘ll', 'against', 'throughout', 'yourselves', 'him', "'d", 'it', 'himself', 'whether', 'move', '’m', 'hereafter', 're', 'while', 'whoever', 'your', 'first', 'amount', 'twelve', 'serious', 'other', 'any', 'off', 'seeming', 'four', 'itself', 'nothing', 'beforehand', 'make', 'out', 'very', 'already', 'various', 'until', 'hers', 'they', 'not', 'them', 'where', 'would', 'since', 'everything', 'at', 'together', 'yet', 'more', 'six', 'back', 'with', 'thereupon', 'becoming', 'around', 'due', 'keep', 'somehow', 'n‘t', 'across', 'all', 'when', 'i', 'empty', 'nine', 'five', 'get', 'see', 'been', 'name', 'between', 'hence', 'ten', 'several', 'from', 'whereupon', 'through', 'hereby', "'ll", 'alone', 'something', 'formerly', 'without', 'above', 'onto', 'except', 'enough', 'become', 'behind', '’d', 'its', 'most', 'n’t', 'might', 'whereas', 'anything', 'if', 'her', 'via', 'fifty', 'is', 'thereby', 'twenty', 'often', 'whereafter', 'their', 'also', 'anyhow', 'cannot', 'our', 'could', 'because', 'who', 'beside', 'by', 'whence', 'being', 'meanwhile', 'this', 'afterwards', 'whenever', 'mostly', 'what', 'one', 'nobody', 'seem', 'less', 'do', '‘d', 'say', 'thus', 'unless', 'along', 'yourself', 'former', 'thru', 'he', 'hundred', 'three', 'sixty', 'me', 'sometime', 'whose', 'you', 'quite', '’ve', 'about', 'even']
nltk = ['ve', 'am', 'can', 'shouldn', 'down', 'from', "wouldn’t", 'as', 'be', 'are', 'too', 'through', "don’t", 'does', 'a', 'but', 'now', 'some', 'an', "couldn’t", 'we', 'below', 'against', 'here', 'won', 'did', 'yourselves', 'was', 'how', 'above', 'him', 'shan', 'it', 'which', "weren’t", 'himself', 'its', 'most', 'the', 'wouldn', 'needn', 're', 'or', 'while', "mustn’t", 'your', 'if', "hasn’t", "shouldn’t", 'yours', 'she', 'her', 'wasn', 'other', 'aren', 'any', 'll', 'off', 'couldn', 'ain', "wasn’t", 'few', 'is', 'of', 'there', "isn’t", 'than', "shan’t", 'why', 'has', 'so', 'in', "didn’t", 'only', 'have', 'itself', 'for', 'under', 'own', 'hasn', 'were', "doesn’t", "needn’t", 'those', 'isn', 'out', 'very', "won’t", 'until', 'hers', 'after', "it’s", 'up', 'they', 'their', 'not', 'doing', 'no', 'them', 'where', 'ourselves', 'themselves', "aren’t", 'our', 'on', 'that', 'theirs', 'nor', 't', 'ours', 'at', 's', 'again', 'same', 'over', 'just', 'ma', 'because', 'who', 'mightn', "you’ve", 'before', 'by', 'more', "should’ve", 'being', 'had', 'weren', 'this', "mightn’t", 'm', 'with', 'should', "haven’t", "hadn’t", 'what', "that’ll", "you’re", 'during', 'haven', 'herself', 'and', 'these', 'such', 'further', 'mustn', 'm', 'with', 'should', "haven't", "hadn't", 'what', "that'll", "you're", 'during', 'haven', 'herself', 'and', 'these', 'such', 'further', 'mustn', 'do', "you'd", 'having', 'didn', 'd', 'hadn', 'y', 'yourself', 'his', 'into', 'once', 'each', 'don', 'all', 'then', 'both', 'when', 'he', 'will', 'me', 'whom', 'i', 'my', 'you', 'to', 'between', 'myself', 'doesn', 'about', 'been', 'o', "you'll", "she's", "'d", "'d", "'s", "'re", "n't", "'d", "'m", "'m", "'m", "n't", "'re", "'ll", "'re", "'ll", "'ve", "n't", "'s", "'s", "'ve", "'ll", "ca", "'ve"]
stop_words = spacy + nltk
stop_words = [re.sub('’', '\'', word) for word in stop_words]
stop_words = list(set(stop_words))

In [4]:
total_features = []
response = requests.get("https://raw.githubusercontent.com/Socram-Occots/proj4-CS598/refs/heads/main/total_features.txt")
for line in response.text.splitlines():
    total_features.append(line.strip())

In [5]:
# load trees
total_trees = []
for i in range(0, 87):
    response = requests.get(f"https://raw.githubusercontent.com/Socram-Occots/proj4-CS598/refs/heads/main/chunk_{i}.json")
    data = json.loads(response.text)
    total_trees += data["chunk"]

In [6]:
ids = []
for i in total_trees:
    ids.append(i['id'])

In [7]:
print(len(ids))

12000


In [8]:
uniqueList = []
duplicateList = []
for i in ids:
    if i not in uniqueList:
        uniqueList.append(i)
    elif i not in duplicateList:
        duplicateList.append(i)

In [9]:
#dont know why there are duplicates but there are
remove_list = []
for i in total_trees:
    if i['id'] in duplicateList:
        remove_list.append(i)
for i in remove_list:
    total_trees.remove(i)

In [10]:
print(len(total_trees))

12000


In [12]:
response = requests.get(f"https://raw.githubusercontent.com/Socram-Occots/proj4-CS598/refs/heads/main/xgboost_class_model_treeless.json")
xgboost_json = json.loads(response.text)

xgboost_json['learner']['gradient_booster']["model"]["trees"] = total_trees

In [13]:
xgboost_json_str = json.dumps(xgboost_json)

In [14]:
final_vectorizer = CountVectorizer(
    preprocessor=lambda x: x.lower(),  # Convert to lowercase
    stop_words=stop_words,             # Remove stop words
    ngram_range=(1, 4),               # Use 1- to 4-grams
    min_df=0.001,                        # Minimum term frequency
    max_df=0.5,                       # Maximum document frequency
    token_pattern=r"\b[\w+\|']+\b", # Use word tokenizer: See Ethan's comment below
    vocabulary=total_features
)

In [17]:
# load xgboost model
temp_file_path = None
with tempfile.NamedTemporaryFile(suffix=".json", delete=False) as temp_file:
    temp_file.write(xgboost_json_str.encode('utf-8'))  # Write JSON string as bytes
    temp_file_path = temp_file.name 

xgmodel = xgboost.XGBClassifier()
xgmodel.load_model(temp_file_path)

In [18]:
xgmodel

XGBClassifier(base_score='5.0016534E-1', booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None,
              feature_types=['int', 'int', 'int', 'int', 'int', 'int', 'int',
                             'int', 'int', 'int', 'int', 'int', 'int', 'int',
                             'int', 'int', 'int', 'int', 'int', '...
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [19]:
# 0.986 needed for each split
for i in range(1, 6):
    test = pd.read_csv(f"F24_Proj3_data\\F24_Proj3_data\\split_{i}\\test.csv")
    test_y = pd.read_csv(f"F24_Proj3_data\\F24_Proj3_data\\split_{i}\\test_y.csv")

    test_X = test["review"]
    test_y = test_y["sentiment"]

    test_X = test_X.str.replace('&lt;.*?&gt;', ' ', regex=True)

    test_X = final_vectorizer.transform(test_X).todense()

    test_pred = xgmodel.predict(test_X)
    print(roc_auc_score(test_y, test_pred))

c:\Users\socra\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:406: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['n'] not in stop_words.
  warnings.warn(


0.9895653945163047
0.9889082154006792
0.9890024090646202
0.98904229698707
0.9893777054095069
